In [1]:

import math                      # providing access to the mathematical functions defined by the C standard
import matplotlib.pyplot as plt  # plotting library
import scipy                     # scientific computnig and technical computing                      # working with, mainly resizing, images
import numpy as np               # dealing with arrays
import glob                      # return a possibly-empty list of path names that match pathname
import os                        # dealing with directories
import pandas as pd              # providing data structures and data analysis tools
import tensorflow as tf       
import itertools
import random
from random import shuffle       # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm            # a nice pretty percentage bar for tasks. Thanks to viewer Daniel Bühler for this suggestion
from PIL import Image
from scipy import ndimage
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
%matplotlib inline
np.random.seed(1)



C:\Users\tasli\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tasli\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tasli\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tasli\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
base_dir ='Monkey'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'validation')



from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping




# Training generator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150,150),
                                                    batch_size=16,
                                           
                                                    shuffle=True,
                                                    class_mode='categorical')

# Test generator
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(test_dir, 
                                                  target_size=(150,150),
                                                  batch_size=16,
                                                 
                                                  shuffle=False,
                                                  class_mode='categorical')

train_num = train_generator.samples
validation_num = validation_generator.samples


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [3]:

# Model Definition----------------------------------------------------------------------------------------------------------------------------------------------
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
____

In [ ]:
LR = 1e-3
height=150
width=150
channels=3
seed=1337
batch_size = 64
num_classes = 10
epochs = 50
data_augmentation = True
num_predictions = 20

filepath=str(os.getcwd()+"/model.h5f")
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# = EarlyStopping(monitor='val_acc', patience=15)
callbacks_list = [checkpoint]#, stopper]

history = model.fit_generator(train_generator,
                              steps_per_epoch= train_num // batch_size,
                              epochs=epochs,
                              validation_data=train_generator,
                              validation_steps= validation_num // batch_size,
                              callbacks=callbacks_list, 
                              verbose = 1
                             )

Epoch 1/50
16/17 [===========================>..] - ETA: 16s - loss: 2.3277 - acc: 0.1367
Epoch 00001: val_acc improved from -inf to 0.12500, saving model to C:\Users\tasli\3D Objects\Deep Learning Project\Monkey_Species_Recognition/model.h5f
17/17 [==============================] - 313s 18s/step - loss: 2.3275 - acc: 0.1287 - val_loss: 2.2942 - val_acc: 0.1250
Epoch 2/50
16/17 [===========================>..] - ETA: 11s - loss: 2.2971 - acc: 0.0938
Epoch 00002: val_acc did not improve from 0.12500
17/17 [==============================] - 216s 13s/step - loss: 2.2968 - acc: 0.0919 - val_loss: 2.2791 - val_acc: 0.0625
Epoch 3/50
16/17 [===========================>..] - ETA: 9s - loss: 2.2902 - acc: 0.1406 
Epoch 00003: val_acc did not improve from 0.12500
17/17 [==============================] - 189s 11s/step - loss: 2.2847 - acc: 0.1544 - val_loss: 2.2435 - val_acc: 0.0781
Epoch 4/50
16/17 [===========================>..] - ETA: 9s - loss: 2.2374 - acc: 0.1602 
Epoch 00004: val_acc imp

Epoch 30/50
16/17 [===========================>..] - ETA: 8s - loss: 1.4317 - acc: 0.4570 
Epoch 00030: val_acc did not improve from 0.59375
17/17 [==============================] - 169s 10s/step - loss: 1.4264 - acc: 0.4559 - val_loss: 1.2776 - val_acc: 0.4062
Epoch 31/50
16/17 [===========================>..] - ETA: 9s - loss: 1.4368 - acc: 0.4336 
Epoch 00031: val_acc did not improve from 0.59375
17/17 [==============================] - 171s 10s/step - loss: 1.4389 - acc: 0.4375 - val_loss: 1.2597 - val_acc: 0.4062
Epoch 32/50
16/17 [===========================>..] - ETA: 8s - loss: 1.2709 - acc: 0.4920 
Epoch 00032: val_acc did not improve from 0.59375
17/17 [==============================] - 167s 10s/step - loss: 1.2482 - acc: 0.5000 - val_loss: 1.1880 - val_acc: 0.4844
Epoch 33/50
16/17 [===========================>..] - ETA: 9s - loss: 1.4140 - acc: 0.4720 
Epoch 00033: val_acc did not improve from 0.59375
17/17 [==============================] - 171s 10s/step - loss: 1.4105 - a

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

# Conclusion
We can clearly see the difference, CNN architecture achieved 98% accuracy as compared to Fully Connected Feed Forward NN architecture which was able to achieve accuracy only up to 45%.

This shows us the importance of different NN architectures for different problems.